In [1]:
"""
  Sofía Almeida Bruno
  https://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits
"""

'\n  Sofía Almeida Bruno\n  https://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits\n'

In [17]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [7]:
# Leemos los datos de los ficheros
train = np.genfromtxt('./datos/optdigits.tra',  delimiter = ",")
test = np.genfromtxt('./datos/optdigits.tes',  delimiter = ",")

In [8]:
X_train = train[:,0:-1]
y_train = train[:,-1]
X_test= test[:,0:-1]
y_test = test[:,-1]

In [9]:
# Preprocesado----------------------------------------------------------

# Añadimos complejidad al modelo
poly = PolynomialFeatures(2)

# Seleccionamos variables utilizando lasso
lasso = LassoCV(n_jobs = -1, max_iter = 55000, verbose = True, cv = 5)

preprocessing = Pipeline(steps=[
    ('poly', poly),
    ('scale',StandardScaler()),
    ('lasso', SelectFromModel(lasso)),
])

X_train = preprocessing.fit_transform(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.0min
..[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished


In [10]:
X_train.shape

(3823, 657)

In [23]:
# Ajuste de parámetros
def adjust_params(X, y, model, params):
    print("------ Grid Search...")
    grid = GridSearchCV(model, params, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')
    grid.fit(X, y)
    print("Mejores parámetros:")
    print(grid.best_params_)
    print("Error CV")
    print(-grid.best_score_)
    return grid.best_estimator_

In [ ]:
lreg = LogisticRegression(tol=0.0001, random_state=123456, solver='saga', max_iter=100, multi_class='multinomial', n_jobs=-1)
params_lreg =  {'penalty':['none','l2'], 'C':[0.5, 1.0, 2.0], 'max_iter':[5000,10000,15000]}

best_lreg = adjust_params(X_train, y_train, lreg, params_lreg)

------ Grid Search...
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
